In [42]:
import numpy as np
import pickle
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

In [9]:
"""
Open the training data.
"""
training_data_filepath = 'F:/ZaknafeinII_Backup_02-02-22/daea/training_data_generation/id16/output/mfcc_training_data_dict.pickle'

with open(training_data_filepath, 'rb') as f:
    data_dict = pickle.load(f)
    
for key in data_dict:
    print(key, 'is of shape', data_dict.get(key).shape)

train_input is of shape (180518, 64)
val_input is of shape (38683, 64)
test_input is of shape (38683, 64)
train_labels is of shape (180518, 64)
val_labels is of shape (38683, 64)
test_labels is of shape (38683, 64)


In [13]:
"""
Standardize/scale the data.
"""

min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler.fit(np.concatenate((data_dict.get('train_input'),
                                   data_dict.get('val_input'),
                                   data_dict.get('test_input'),
                                   data_dict.get('train_labels'),
                                   data_dict.get('val_labels'),
                                   data_dict.get('test_labels'))))

overlap_train_scaled = min_max_scaler.transform(data_dict.get('train_input'))
overlap_val_scaled = min_max_scaler.transform(data_dict.get('val_input'))
overlap_test_scaled = min_max_scaler.transform(data_dict.get('test_input'))
character_train_scaled = min_max_scaler.transform(data_dict.get('train_labels'))
character_val_scaled = min_max_scaler.transform(data_dict.get('val_labels'))
character_test_scaled = min_max_scaler.transform(data_dict.get('test_labels'))

In [21]:
"""
Calculate the ideal masks such that (input vals)*(mask coefficients)=(label vals)
So --> (mask coefficients)=(label vals)/(input vals)
"""
# zero_count = 0
# minimum = -1
# for i in overlap_train_scaled:
#     for j in i:
#         if j == 0:
#             zero_count = zero_count + 1
#         if minimum == -1 and j != 0:
#             minimum = j
#         if j < minimum and j != 0:
#             minimum = j
# print(zero_count, minimum)

imask_train = np.divide(character_train_scaled, overlap_train_scaled)
imask_val = np.divide(character_val_scaled, overlap_val_scaled)
imask_test = np.divide(character_test_scaled, overlap_test_scaled)

C:\Users\ZAKNAF~1\AppData\Local\Temp/ipykernel_10456/438024550.py:17: RuntimeWarning: divide by zero encountered in true_divide
  imask_train = np.divide(character_train_scaled, overlap_train_scaled)
C:\Users\ZAKNAF~1\AppData\Local\Temp/ipykernel_10456/438024550.py:18: RuntimeWarning: divide by zero encountered in true_divide
  imask_val = np.divide(character_val_scaled, overlap_val_scaled)
C:\Users\ZAKNAF~1\AppData\Local\Temp/ipykernel_10456/438024550.py:19: RuntimeWarning: divide by zero encountered in true_divide
  imask_test = np.divide(character_test_scaled, overlap_test_scaled)
C:\Users\ZAKNAF~1\AppData\Local\Temp/ipykernel_10456/438024550.py:19: RuntimeWarning: invalid value encountered in true_divide
  imask_test = np.divide(character_test_scaled, overlap_test_scaled)


In [39]:
"""
Given overlap, predict imask.
"""

print('input shape:', overlap_train_scaled.shape)
print('input contains NaN values?:', np.any(np.isnan(overlap_train_scaled)))
print('input contains inf values?:', np.any(np.isinf(overlap_train_scaled)))
print('input datatype:', overlap_train_scaled.dtype)
overlap_train_scaled = overlap_train_scaled.astype('float32')
print('output shape:', imask_train.shape)
print('output contains NaN values?:', np.any(np.isnan(imask_train)))
print('output contains inf values?:', np.any(np.isinf(imask_train)))
imask_train[np.isinf(imask_train)] = 0
print('output contains inf values?:', np.any(np.isinf(imask_train)))


input shape: (180518, 64)
input contains NaN values?: False
input contains inf values?: False
input datatype: float32
output shape: (180518, 64)
output contains NaN values?: False
output contains inf values?: True
output contains inf values?: False


LinearRegression()

In [ ]:
"""
Linear Regression
"""

model_list = [LinearRegression(), KNeighborsRegressor(), DecisionTreeRegressor()]

for model_type in model_list:
    # define model
    model = model_type
    # fit model
    model.fit(overlap_train_scaled, imask_train)

    x = overlap_test_scaled[0]
    y = imask_test[0]
    yhat = model.predict([x])[0]

    x_axis_list = list(range(0, len(x)))
    plt.plot(x_axis_list, y)
    plt.plot(x_axis_list, yhat)
    plt.title(str(model_type))
    plt.show()

In [ ]:
"""
Knei
"""

# define model
model = KNeighborsRegressor()
# fit model
model.fit(overlap_train_scaled, imask_train)

x = overlap_test_scaled[0]
y = imask_test[0]
yhat = model.predict([x])[0]

x_axis_list = list(range(0, len(x)))
plt.plot(x_axis_list, y)
plt.plot(x_axis_list, yhat)
plt.title('Linear Regression')
plt.show()